# Инструменты для работы с языком 

## Задача: классификация твитов по тональности

У нас есть датасет из твитов, про каждый указано, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску.

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [1]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv


--2022-06-05 05:06:42--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-05 05:06:43--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd28ef0439215423a6f607d43ea.dl.dropboxusercontent.com/cd/0/inline/BmlEM3VrNOm1G_88owmCcgeo93hTv0msvh-PBVoZ3S6awGHjMC-CRwJgEGKcRUDZmRzSbeHOuhiSI_zkvB0yptAqiMNGPyFOH-o9VUMKXTK4pnfS5CYdkIZBlgO-XbwhcCpUzWtgUsRyIRNJEEVWrAu_wh0OjiDOi1U3l-QHuTCPbA/file# [following]
--2022-06-05 05:06:43--  https://ucd28ef0439215423a6f607d43ea.dl.dropboxusercontent.com/cd/0/inline/BmlEM3VrNOm1G_88owmCcgeo93hTv0msvh-PBVoZ3S6awGHjMC-CRwJgEGKcRUDZmRzS

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [6]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 0 to 111922
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 5.2+ MB


In [8]:
df['label'].value_counts()

positive    114911
negative    111923
Name: label, dtype: int64

In [9]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], random_state = 42)

In [10]:
type(x_train[0])

pandas.core.series.Series

In [11]:
y_train.value_counts()

positive    86310
negative    83815
Name: label, dtype: int64

## Стоп-слова и пунктуация

*Стоп-слова* -- это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [12]:
# у вас здесь, вероятно, выскочит ошибка и надо будет загрузить стоп слова (в тексте ошибки написано, как)
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'бол

In [13]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
noise = stopwords.words('russian') + list(punctuation)

В векторизаторах за стоп-слова, логичным образом, отвечает аргумент `stop_words`.

In [15]:
"""
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))
"""

'\nvec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=noise)\nbow = vec.fit_transform(x_train)\nclf = LogisticRegression(random_state=42)\nclf.fit(bow, y_train)\npred = clf.predict(vec.transform(x_test))\nprint(classification_report(pred, y_test))\n'

Получилось чууть лучше. Что ещё можно сделать?

# NN

In [34]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout, TextVectorization, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
from sklearn import model_selection, preprocessing, linear_model

In [18]:
x_train

95791     Хватит писать такие глупости!!надо правильно р...
63858     Отдохнуть от  дел наших скорбных можно здесь! ...
77643     Когда забываю ipod , спасает @pubblartnanesmo ...
7876      Ура я дома !!! Чертовы понедельники, ненавижу ...
62141     Дорогой Фло, я совершила-таки первую в жизни п...
                                ...                        
4968      Хочу, чтобы графический планшетик скорее прише...
103694    RT @OmenDougnter_: мы с @fluffy_irk @hamsterka...
17021     дому чуть больше 30 лет а мне приходится втору...
31956     @MIKEFUCKINGWAY @_im_killjoy_ даже представить...
7047      через 8 дней будет месяц как мы не общаемся.. ...
Name: text, Length: 170125, dtype: object

In [19]:
type(x_train[0].astype('string'))

pandas.core.series.Series

In [20]:
type(x_train[0])

pandas.core.series.Series

In [21]:
# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [22]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [23]:
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [24]:
for raw in train_data.take(1):
    pass

In [25]:
# функция меняет кодировку на бинарную! https://stackoverflow.com/questions/69756755/tensorflow-unicode-text-encoding-decoding
raw

(<tf.Tensor: shape=(16,), dtype=string, numpy=
 array([b'\xd0\xa5\xd0\xb2\xd0\xb0\xd1\x82\xd0\xb8\xd1\x82 \xd0\xbf\xd0\xb8\xd1\x81\xd0\xb0\xd1\x82\xd1\x8c \xd1\x82\xd0\xb0\xd0\xba\xd0\xb8\xd0\xb5 \xd0\xb3\xd0\xbb\xd1\x83\xd0\xbf\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8!!\xd0\xbd\xd0\xb0\xd0\xb4\xd0\xbe \xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xbe \xd1\x80\xd0\xb5\xd1\x87\xd1\x8c \xd1\x84\xd0\xbe\xd1\x80\xd0\xbc\xd1\x83\xd0\xbb\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c!!!(((',
        b'\xd0\x9e\xd1\x82\xd0\xb4\xd0\xbe\xd1\x85\xd0\xbd\xd1\x83\xd1\x82\xd1\x8c \xd0\xbe\xd1\x82  \xd0\xb4\xd0\xb5\xd0\xbb \xd0\xbd\xd0\xb0\xd1\x88\xd0\xb8\xd1\x85 \xd1\x81\xd0\xba\xd0\xbe\xd1\x80\xd0\xb1\xd0\xbd\xd1\x8b\xd1\x85 \xd0\xbc\xd0\xbe\xd0\xb6\xd0\xbd\xd0\xbe \xd0\xb7\xd0\xb4\xd0\xb5\xd1\x81\xd1\x8c!  :-) \nhttp://t.co/960kujVLax',
        b'\xd0\x9a\xd0\xbe\xd0\xb3\xd0\xb4\xd0\xb0 \xd0\xb7\xd0\xb0\xd0\xb1\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x8e ipod , \xd1\x81\xd0\xbf\xd0\xb0

Проверка содержания датасета:

In [44]:
tf.compat.as_str(raw[0][0].numpy())

'Хватит писать такие глупости!!надо правильно речь формулировать!!!((('

In [45]:
raw[0][0].numpy().decode(encoding='utf-8')

'Хватит писать такие глупости!!надо правильно речь формулировать!!!((('

In [26]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [28]:
vocab_size = 10000
seq_len = 100

vectorize_layer = TextVectorization(
    #standardize=custom_standartization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [30]:
embedding_dim=20

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name="embedding"),
    Conv1D(20, (3)),
    Conv1D(20, (2)),
    GRU(30),
    #GlobalAveragePooling1D(),
    Dense(100, activation='relu'),
    Dense(1)
])

In [31]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [32]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
10633/10633 [==============================] - 125s 10ms/step - loss: 0.6932 - accuracy: 0.4927 - val_loss: 0.6931 - val_accuracy: 0.4957
Epoch 2/5
10633/10633 [==============================] - 102s 10ms/step - loss: 0.6931 - accuracy: 0.4927 - val_loss: 0.6931 - val_accuracy: 0.4957
Epoch 3/5
10633/10633 [==============================] - 94s 9ms/step - loss: 0.6931 - accuracy: 0.4927 - val_loss: 0.6931 - val_accuracy: 0.4957


KeyboardInterrupt: ignored

На таком датасете GRU обучаться не хочет

In [46]:
embedding_dim = 20
model_NN = Sequential([
                    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_len),
                    Flatten(),
                    Dense(10, activation='relu'),
                    Dense(1, activation='sigmoid')
])
   
model_NN.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_NN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 20)           200000    
                                                                 
 flatten (Flatten)           (None, 2000)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                20010     
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 220,021
Trainable params: 220,021
Non-trainable params: 0
_________________________________________________________________


In [47]:
history = model.fit(x_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(x_test, y_test),
                    batch_size=16)

Epoch 1/5
10633/10633 [==============================] - 113s 10ms/step - loss: 0.6931 - accuracy: 0.4927 - val_loss: 0.6931 - val_accuracy: 0.4957
Epoch 2/5
10633/10633 [==============================] - 113s 11ms/step - loss: 0.6931 - accuracy: 0.4927 - val_loss: 0.6932 - val_accuracy: 0.4957
Epoch 3/5
10633/10633 [==============================] - 128s 12ms/step - loss: 0.6931 - accuracy: 0.4927 - val_loss: 0.6932 - val_accuracy: 0.4957
Epoch 4/5
 1108/10633 [==>...........................] - ETA: 1:29 - loss: 0.6932 - accuracy: 0.4992

KeyboardInterrupt: ignored

На полносвязной та же песня. Надо менять подход:

In [48]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

vocab_size = 10000
seq_len = 100

tokenizer = Tokenizer(num_words=vocab_size )
tokenizer.fit_on_texts(df.text)
sequences = tokenizer.texts_to_sequences(df.text)
tweets = pad_sequences(sequences, maxlen=seq_len)
print(tweets)

[[   0    0    0 ... 3340 2953  527]
 [   0    0    0 ...  178  121   20]
 [   0    0    0 ... 3903   26   40]
 ...
 [   0    0    0 ...  221   27   53]
 [   0    0    0 ...    1 2354 7255]
 [   0    0    0 ... 5472   16   36]]


In [50]:
labels = encoder.fit_transform(df['label'])

In [51]:
x_train, x_test, y_train, y_test = train_test_split(tweets, labels)

In [52]:
from keras.models import Sequential
from keras import layers

embedding_dim = 20
model = Sequential([
                    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_len),
                    Flatten(),
                    Dense(10, activation='relu'),
                    Dense(1, activation='sigmoid')
])
   
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 20)           200000    
                                                                 
 flatten_1 (Flatten)         (None, 2000)              0         
                                                                 
 dense_4 (Dense)             (None, 10)                20010     
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 220,021
Trainable params: 220,021
Non-trainable params: 0
_________________________________________________________________


In [53]:
history = model.fit(x_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(x_test, y_test),
                    batch_size=16)

Epoch 1/5
10633/10633 [==============================] - 48s 4ms/step - loss: 0.4978 - accuracy: 0.7461 - val_loss: 0.4708 - val_accuracy: 0.7688
Epoch 2/5
10633/10633 [==============================] - 42s 4ms/step - loss: 0.4205 - accuracy: 0.8020 - val_loss: 0.4806 - val_accuracy: 0.7649
Epoch 3/5
10633/10633 [==============================] - 49s 5ms/step - loss: 0.3553 - accuracy: 0.8406 - val_loss: 0.5049 - val_accuracy: 0.7583
Epoch 4/5
10633/10633 [==============================] - 46s 4ms/step - loss: 0.2976 - accuracy: 0.8713 - val_loss: 0.5716 - val_accuracy: 0.7519
Epoch 5/5
10633/10633 [==============================] - 48s 4ms/step - loss: 0.2531 - accuracy: 0.8929 - val_loss: 0.6514 - val_accuracy: 0.7448


Сеть пошла в переобучение, но работает. 